#  🛠 **Step-by-Step Autogen Guide**

## 📦 **1. Installation**

To get started with `Autogen`, ensure you have the necessary

In [ ]:
%pip install pyautogen~=0.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 9.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


## 📘 **2. Import Libraries**

After the installation, pull in the `autogen` module.

In [ ]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4"],
    },
)

## ⚙️ **3. Configuration**

Follow these steps to set up your configuration:

- **Configuration List**: Define your list based on your requirements.

## 🤖 **4. Initialize Agents**

### 🔧 **a. AssistantAgent**

Set up your `AssistantAgent` with the appropriate parameters.

In [ ]:
# create an AssistantAgent named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "seed": 42,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0,  # temperature for sampling
    },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
)

### 🔧 **b. UserProxyAgent**

Proceed to set up the `UserProxyAgent` and initiate a chat session.

In [ ]:
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,  # set to True or image name like "python:3" to use docker
    },
)

In [ ]:
# the assistant receives a message from the user_proxy, which contains the task description

user_proxy.initiate_chat(
    assistant,
    message="So I want a code that browss the internet and gives me to news in ai."
)

user_proxy (to assistant):

So I want a code that browss the internet and gives me to news in ai.

--------------------------------------------------------------------------------
assistant (to user_proxy):

Sure, I can help with that. We can use Python with the BeautifulSoup and requests libraries to scrape news from a website that publishes AI news. 

Here's a simple script that fetches the latest news from the "AI" section of the "Technology" category on the BBC News website. This script will print the titles and URLs of the latest news articles.

Please note that web scraping should be done in compliance with the terms of service of the website. Also, the structure of the website can change over time, so the script might not work in the future if BBC changes their website design.

```python
# filename: ai_news_scraper.py

import requests
from bs4 import BeautifulSoup

def get_ai_news():
    url = "https://www.bbc.co.uk/search?q=artificial+intelligence"
    response = requests.get(u

KeyboardInterrupt: ignored

## 💻 **5. IPython Magic**

For seamless integration with IPython, consider using the `IPythonUserProxyAgent` class.

In [ ]:
from typing import Dict, Union
from IPython import get_ipython

class IPythonUserProxyAgent(autogen.UserProxyAgent):
    def __init__(self, name: str, **kwargs):
        super().__init__(name, **kwargs)
        self._ipython = get_ipython()

    def generate_init_message(self, *args, **kwargs) -> Union[str, Dict]:
        return super().generate_init_message(*args, **kwargs) + """
If you suggest code, the code will be executed in IPython."""

    def run_code(self, code, **kwargs):
        result = self._ipython.run_cell("%%capture --no-display cap\n" + code)
        log = self._ipython.ev("cap.stdout")
        log += self._ipython.ev("cap.stderr")
        if result.result is not None:
            log += str(result.result)
        exitcode = 0 if result.success else 1
        if result.error_before_exec is not None:
            log += f"\n{result.error_before_exec}"
            exitcode = 1
        if result.error_in_exec is not None:
            log += f"\n{result.error_in_exec}"
            exitcode = 1
        return exitcode, log, None

ipy_user = IPythonUserProxyAgent(
    "ipython_user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE") or x.get("content", "").rstrip().endswith('"TERMINATE".'),
)

In [ ]:
# the assistant receives a message from the user, which contains the task description
ipy_user.initiate_chat(
    assistant,
    message="""figure out a way to get all relevent news about what is happening in the world of ai, I want this code to run daily  and from the news it will give me like a 5 bullet point summary. Specialized AI News Sites: Websites like AI News, Synced Review, or VentureBeat's AI section are dedicated to AI and machine learning news. These sites are likely to have a wealth of relevant articles.

General Tech News Sites with AI Sections: Websites like TechCrunch, Wired, MIT Technology Review, and The Verge often have specific sections for AI-related news. These sections can be targeted specifically for scraping.

Academic and Research Publications: Websites of institutions like MIT, Stanford, or dedicated AI research platforms often publish news and updates on AI research.

Legal Compliance: Before scraping, ensure you're allowed to do so by checking each website's robots.txt file and terms of service. Keep in mind we do now want to use any APIs for NEWS
    """,
)

ipython_user_proxy (to assistant):

figure out a way to get all relevent news about what is happening in the world of ai, I want this code to run daily  and from the news it will give me like a 5 bullet point summary. Specialized AI News Sites: Websites like AI News, Synced Review, or VentureBeat's AI section are dedicated to AI and machine learning news. These sites are likely to have a wealth of relevant articles.

General Tech News Sites with AI Sections: Websites like TechCrunch, Wired, MIT Technology Review, and The Verge often have specific sections for AI-related news. These sections can be targeted specifically for scraping.

Academic and Research Publications: Websites of institutions like MIT, Stanford, or dedicated AI research platforms often publish news and updates on AI research.

Legal Compliance: Before scraping, ensure you're allowed to do so by checking each website's robots.txt file and terms of service. Keep in mind we do now want to use any APIs for NEWS
    
If yo

ZeroDivisionError: ignored

ipython_user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
Found 0 paragraphs on https://www.venturebeat.com/category/ai/
Found 0 paragraphs on https://techcrunch.com/artificial-intelligence/
Found 25 paragraphs on https://www.wired.com/category/artificial-intelligence/
Website: https://www.wired.com/category/artificial-intelligence/
Paragraph: To revisit this article, visit My Profile, then View saved stories.
Summary: 


Website: https://www.wired.com/category/artificial-intelligence/
Paragraph: To revisit this article, select My Account, then View saved stories
Summary: 


Website: https://www.wired.com/category/artificial-intelligence/
Paragraph: By Fernanda Seavon
Summary: 


Website: https://www.wired.com/category/artificial-intelligence/
Paragraph: By Justin Pot
Summary: 


Website: https://www.wired.com/category/artificial-intelligence/
Paragraph: By Sarah Gundle, PsyD
Summary: 


Website: https://www.wired.com/category/artificial-intelligence/
Paragrap

ZeroDivisionError: ignored

ipython_user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
Found 0 divs on https://www.venturebeat.com/category/ai/
Found 99 divs on https://techcrunch.com/artificial-intelligence/
Website: https://techcrunch.com/artificial-intelligence/
Div: 




TechCrunch


















plus-bold








Summary: 


Website: https://techcrunch.com/artificial-intelligence/
Div: 



TechCrunch


















plus-bold







Summary: 


Website: https://techcrunch.com/artificial-intelligence/
Div: 


TechCrunch

















Summary: 


Website: https://techcrunch.com/artificial-intelligence/
Div: 
plus-bold






Summary: 


Website: https://techcrunch.com/artificial-intelligence/
Div: 





TechCrunch


Open Navigation






TechCrunch






















AI





				Why this autonomous vehicle veteran joined a legged robotics startup			




		Nov 06, 2023	

Rita Liao 




		Zhang Li‘s career path has looked like a bellwether for China’s tech trends. When the Cisc

...

[autogen.oai.completion: 11-07 04:38:41] {252} INFO - retrying in 10 seconds...
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/autogen/oai/completion.py", line 224, in _get_response
    response = openai_completion.create(request_timeout=request_timeout, **config)
  File "/usr/local/lib/python3.10/dist-packages/openai/api_resources/chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/openai/api_resources/abstract/engine_api_resource.py", line 155, in create
    response, _, api_key = requestor.request(
  File "/usr/local/lib/python3.10/dist-packages/openai/api_requestor.py", line 299, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "/usr/local/lib/python3.10/dist-packages/openai/api_requestor.py", line 710, in _interpret_response
    self._interpret_response_line(
  File "/usr/local/lib/python3.10/dist-packages/openai/api_requestor.py", lin

INFO:autogen.oai.completion:retrying in 10 seconds...
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/autogen/oai/completion.py", line 224, in _get_response
    response = openai_completion.create(request_timeout=request_timeout, **config)
  File "/usr/local/lib/python3.10/dist-packages/openai/api_resources/chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/openai/api_resources/abstract/engine_api_resource.py", line 155, in create
    response, _, api_key = requestor.request(
  File "/usr/local/lib/python3.10/dist-packages/openai/api_requestor.py", line 299, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "/usr/local/lib/python3.10/dist-packages/openai/api_requestor.py", line 710, in _interpret_response
    self._interpret_response_line(
  File "/usr/local/lib/python3.10/dist-packages/openai/api_requestor.py", line 775, in _interpret_respo

[autogen.oai.completion: 11-07 04:38:51] {252} INFO - retrying in 10 seconds...
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/autogen/oai/completion.py", line 224, in _get_response
    response = openai_completion.create(request_timeout=request_timeout, **config)
  File "/usr/local/lib/python3.10/dist-packages/openai/api_resources/chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/openai/api_resources/abstract/engine_api_resource.py", line 155, in create
    response, _, api_key = requestor.request(
  File "/usr/local/lib/python3.10/dist-packages/openai/api_requestor.py", line 299, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "/usr/local/lib/python3.10/dist-packages/openai/api_requestor.py", line 710, in _interpret_response
    self._interpret_response_line(
  File "/usr/local/lib/python3.10/dist-packages/openai/api_requestor.py", lin

INFO:autogen.oai.completion:retrying in 10 seconds...
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/autogen/oai/completion.py", line 224, in _get_response
    response = openai_completion.create(request_timeout=request_timeout, **config)
  File "/usr/local/lib/python3.10/dist-packages/openai/api_resources/chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/openai/api_resources/abstract/engine_api_resource.py", line 155, in create
    response, _, api_key = requestor.request(
  File "/usr/local/lib/python3.10/dist-packages/openai/api_requestor.py", line 299, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "/usr/local/lib/python3.10/dist-packages/openai/api_requestor.py", line 710, in _interpret_response
    self._interpret_response_line(
  File "/usr/local/lib/python3.10/dist-packages/openai/api_requestor.py", line 775, in _interpret_respo

RateLimitError: ignored